In [ ]:
!pip install --user psycopg2-binary
!pip install --user sql_magic

import sys
import pprint
import psycopg2

In [ ]:
# Copy these settings from https://platform.adobe.com/query/configuration
connection = psycopg2.connect(
    sslmode='require',
     host='experienceplatform.platform-query.adobe.io',
     port='80',
     dbname='prod:all',
     user='907075E95BF479EC0A495C73@AdobeOrg',
     password='eNqrVqowLVWyUsrMLY7PSzTUzU6t1DXUS04tUtJRSsxJB8oEBRuZminVAgALngwl.eNplU21v2jAQ_i_52JFiO7aTVJq0lJSXQaBQWpYKKXJsh5pCHCWmK63632dStEmrP5x8j-_1ufO7o4Rz5UASeCEGVoQAZFAKQHMMXUQZdbEokBsAz4qcIFowIUIisoOETsfhOyVLk7VBGK-yg8qqHTOFrvf29dDI-vMNRT7tU0IIDf2eF_X9PgAAAvgjEjqXo9gaN4YZaU3f104jm0bpcu1crZ0nY6rmqttV-8YtGbxkJ4ed3qjykuv9Ce6ezbsvsJtuNyRdLZ7S5cik2_5T0gMgiW_gZHWDHgcWW02fZ4P0bbZKjsn-UU3uwHG-5GixTeF0eY9mSxEnoD-YgvR3EkevSdyP5qvFS7p_-LmI1s6HLdQcK9m2y23ezOhnWVqUNRY7F2nVYmPV-9EoGg-T5cPk1qdDOu7NUm_uz6M7_L09p67_8k8ohTQIsZfJAEOJc0t4gYSLOcEu48JzARGY5sTnVNIz_7X5Mj-Y-SIEIUHQJYEsXOz5yGU-yl0_Z9LHgQ8CHJz9NbfetSw37MLWXV9AH3Dm5YiGxUXsx4u7Zbr4NUSRnZ2_HE7HCIYwHrSJJftMTCABuE1s4b1WFiUhRdJj_LQgVr0-TpojC2nUO6hjNfimw_H1bNO2L18rVcsmU6W1CygGp3OiheuWZCtLJTrnCXdqyUSm6w0r1RszFmk6dh3V6cZ2NkihL6tabyU3UtzWWhy46enSyFfzxa7WO2m9D8IuMJd7VrKNXVZWqc6_leQ2n42UMfP_H3E-_gBLufnI.eNoBAAH__jKkqFTyFP-Oo56QmY-1Prm2MjvtaqbZuZfvEBZbKS5NlpC53Yf4jy2Xqjtgv6EJgPLsGhJo0CAxOt6wikswlFYypSaWAAQEuhvcnSEyQNv0sxaZD627wfOBWlzebquNujogZITTF5d_qNm33zq52v0WBf09PE4rVW8ps9Xo9pJ7FSmR1u155i8yZaRtXFZrS2cD3R17Aved9GAITY57HoIv006BrhoP0xLgbAszp8n0ey2_EQ8sUXLAuFJpelJE_9gvaqGYCnJu7N5_X0VDhedf8GPNNvoVO80KR9oyiEwDUf1k8rxrCZ-nNwKR7YiOddgGzCEwO5Nmt5kNY4nb2HLwJ3ub')

In [ ]:
# Enable sql magic
%load_ext sql_magic
%config SQL.conn_name = 'connection'

In [ ]:
%%read_sql
show tables

In [ ]:
%%read_sql df_result
SELECT _experienceplatform.identification.ecid as ecid,
_experienceplatform.carinsurance.insuranceKm as km,
                     _experienceplatform.carinsurance.insuranceCarType as cartype,
                     _experienceplatform.carinsurance.insuranceAge as age,
                     _experienceplatform.carinsurance.insuranceGender as gender,
                     _experienceplatform.carinsurance.insuranceCarBrand as carbrand,
                     _experienceplatform.carinsurance.insuranceLeasing as leasing,
                     _experienceplatform.carinsurance.insuranceCity as city,
                    _experienceplatform.carinsurance.insuranceCountry as country,
                     _experienceplatform.carinsurance.insuranceNationality as nationality,
                    _experienceplatform.carinsurance.insurancePrimaryDriver as primaryuser,
                     _experienceplatform.carinsurance.insurancePurchase as purchase,
                    _experienceplatform.carinsurance.insuranceBasicPrice as pricequote,
                    timestamp
FROM emea_car_insurance_quotes_rtml_api

In [ ]:
df_result.head()

In [ ]:
train_config_properties = { "n_estimators": "80",
                            "max_depth": "5"
                            }

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import time, timedelta
import os

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import time, timedelta
import os
#from .utils import get_client_context

def train_load(config_properties, data):
    print("Training Data Load Start")

    #########################################
    # Load Data
    #########################################    
    print('reading training data')
    df1 = data

    df1 = df1[['ecid', 'km', 'cartype', 'age', 'gender', 'carbrand', 'leasing', 'city', 
           'country', 'nationality', 'primaryuser', 'purchase', 'pricequote', 'timestamp']]
    
    #########################################
    # Data Rollup
    ######################################### 
    df1['timestamp'] = pd.to_datetime(df1.timestamp)
    df1['hour'] = df1['timestamp'].dt.hour.astype(int)
    df1['dayofweek'] = df1['timestamp'].dt.dayofweek
    
    df1.loc[(df1['purchase'] == 'yes'), 'purchase'] = 1
    df1.purchase.fillna(0, inplace=True)
    df1['purchase'] = df1['purchase'].astype(int)
    
    df1.dropna(subset = ['ecid'], inplace=True)
    df1['ecid'] = df1['ecid'].astype(str)
    df_conv_dict = df1.groupby('ecid').max()[['purchase']]
    df_conv_dict = df_conv_dict.to_dict()
    
    idx = df1.groupby('ecid')['timestamp'].transform(max) == df1['timestamp']
    df1 = df1[idx]
    df1 = df1.drop_duplicates(subset = 'ecid', keep = 'last', inplace = False)
    df1['purchase'] = df1['ecid'].map(df_conv_dict['purchase'])
    
    #########################################
    # Data Preparation/Feature Engineering
    #########################################      
    
    df1['carbrand'] = df1['carbrand'].str.lower()
    df1['country'] = df1['country'].str.lower()
    df1.loc[(df1['carbrand'] == 'vw'), 'carbrand'] = 'volkswagen'
    df1.loc[(df1['carbrand'] == 'citroen'), 'carbrand'] = 'cadillac'
    df1.loc[(df1['carbrand'] == 'opel'), 'carbrand'] = 'bmw'
    df1.loc[(df1['carbrand'] == 'mini'), 'carbrand'] = 'volkswagen'
    
    df1.loc[(df1['cartype'] == 'SUV / Geländewagen') | (df1['cartype'] == 'SUV / Tout terrain'), 'cartype'] = 'suv'
    df1.loc[(df1['cartype'] == 'Kleinwagen') | (df1['cartype'] == 'Untere Mittelklasse') | (df1['cartype'] == 'Mikroklasse'), 'cartype'] = 'hatchback'
    df1.loc[(df1['cartype'] == 'Mittelklasse') | (df1['cartype'] == 'Obere Mittelklasse'), 'cartype'] = 'sedan'
    df1.loc[(df1['cartype'] == 'Kompaktvan / Minivan'), 'cartype'] = 'minivan'
    df1.loc[(df1['cartype'] == 'Cabriolet / Roadster'), 'cartype'] = 'convertible'
    df1.loc[(df1['cartype'] == 'Coupé / Sportwagen'), 'cartype'] = 'coupe'
    df1.loc[(df1['cartype'] == 'dataLayerNull'), 'cartype'] = pd.np.nan
    df1.loc[(df1['cartype'] == 'Luxusklasse'), 'cartype'] = 'luxury'
    df1.loc[(df1['cartype'] == 'Strasse'), 'cartype'] = 'mpv'
    
    df1.loc[(df1['leasing'] == 'FALSE'), 'leasing'] = 'no'
    df1.loc[df1['country'] == 'at', 'country'] = 'austria'
    df1.loc[(df1['leasing'] == 'dataLayerNull'), 'leasing'] = pd.np.nan
    df1.loc[(df1['gender'] == 'dataLayerNull'), 'gender'] = pd.np.nan
    df1.loc[(df1['carbrand'] == 'dataLayerNull'), 'carbrand'] = pd.np.nan

    df1['age'].fillna(df1['age'].median(), inplace=True)
    df1['gender'].fillna('notgiven', inplace=True)
    df1['leasing'].fillna('notgiven', inplace=True)
    df1['carbrand'].fillna('bmw', inplace=True)
    df1['country'].fillna('germany', inplace=True)
    df1['cartype'].fillna('na', inplace=True)
    df1['primaryuser'].fillna('na', inplace=True)
    df1['nationality'].fillna('na', inplace=True)
    df1['km'].fillna('na', inplace=True)
    
    df1['city'] = df1.groupby('country')['city'].transform(lambda x : x.fillna(x.mode()))
    df1.dropna(subset = ['pricequote'], inplace=True)
    
    #grouping
    grouping_cols = ['carbrand', 'cartype', 'city', 'country']
    
    for col in grouping_cols:
        df_idx = pd.DataFrame(df1[col].value_counts().head(6))

        def grouping(x):
            if x in df_idx.index:
                return x
            else:
                return "Others"
        df1[col] = df1[col].apply(lambda x: grouping(x))
        
    def age(x):
        if x < 20:
            return "u20"
        elif x > 19 and x < 29:
            return "20-28"
        elif x > 28 and x < 43:
            return "29-42"
        elif x > 42 and x < 55:
            return "43-54"
        elif x > 54 and x < 65:
            return "55-64"
        elif x >= 65: 
            return "65+"
        else: 
            return "Others"
        
    df1['age'] = df1['age'].astype(int)
    df1['age_bucket'] = df1['age'].apply(lambda x: age(x))
    
    df_final = df1[['hour', 'dayofweek','age_bucket', 'gender', 'city',  
       'country', 'carbrand', 'cartype', 'leasing', 'pricequote', 'purchase']]
    
    cat_cols = ['age_bucket', 'gender', 'city', 'dayofweek', 'country', 'carbrand', 'cartype', 'leasing']
    df_final = pd.get_dummies(df_final, columns = cat_cols)
    
    dataframe = df_final.copy()
    
    print("Training Data Load Finish")
    
    return dataframe

In [ ]:
df_final = train_load(train_config_properties, df_result)

## Split data

In [ ]:
from sklearn.model_selection import train_test_split
train_set, val = train_test_split(df_final, test_size = 0.2, random_state = 32)

In [ ]:
train_x = train_set.drop('purchase', axis=1)
train_feats = train_x.columns.tolist()
print("Totals Features: ",len(train_feats))

## Model Training

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import os
import tarfile
import subprocess

def underSample(data):
    conv = data[data['purchase'] > 0]
    non_conv = data[data['purchase'] == 0]
    sample_size = len(conv)
    non_sample = non_conv.sample(n = sample_size)
    frames = [conv, non_sample]
    result = pd.concat(frames)
    return result

class RandomForest():
    def __init__(self, config_properties, feat_len):
        print("initiating model")
        self.n_estimators = int(config_properties['n_estimators'])
        self.max_depth = int(config_properties['max_depth'])
        self.feature_len = feat_len
        self.model = RandomForestClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, random_state=32)

    def make_tarfile(self, output_filename, source_dir):
        with tarfile.open(output_filename, "w:gz") as tar:
            tar.add(source_dir, arcname=os.path.basename(source_dir))
    
    def generate_onnx_resources(self):        
        install_dir = os.path.expanduser('~/my-workspace')
        print("Generating Onnx")
        try:
            subprocess.check_call(["conda", "uninstall", "-y", "protobuf"])
        except:
            print("protobuf not installed via conda")
        
        subprocess.check_call(["python", '-m', 'pip', 'install', 'skl2onnx'])
        
        from skl2onnx import convert_sklearn
        from skl2onnx.common.data_types import FloatTensorType
        
        # ONNX-ification
        initial_type = [('float_input', FloatTensorType([None, self.feature_len]))]

        print("Converting Model to Onnx")
        onx = convert_sklearn(self.model, initial_types=initial_type)
             
        with open("model_new.onnx", "wb") as f:
            f.write(onx.SerializeToString())
            
        self.make_tarfile('model_new.tar.gz', 'model_new.onnx')
        print("Model onnx created")
        
def train(config_properties, data):

    print("Train Start")
      
    # UnderSampling
    #result = underSample(data)
    y_train = data['purchase']
    X_train = data.drop('purchase', axis=1)
    feat_len = len(X_train.columns)
    
    # Fit model
    lead_score = RandomForest(config_properties, feat_len)
    print("fitting model")
    model = lead_score.model
    print(X_train.shape)
    model.fit(X_train, y_train)
    lead_score.generate_onnx_resources()
    print("Train Complete")
    return lead_score

In [ ]:
train(train_config_properties, train_set)

## Test Onnx Runtime

In [ ]:
!pip install onnxruntime

In [ ]:
val_x = val.drop('purchase', axis=1)
test_list = val_x.head().values.tolist()
print(len(test_list[0]))

In [ ]:
import numpy as np
X_test = np.array(test_list)

In [ ]:
import onnxruntime as rt
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument

sess = rt.InferenceSession("model_new.onnx")
input_name = sess.get_inputs()[0].name
prob_name = sess.get_outputs()[1].name
try:
    pred_onx = sess.run([prob_name],{input_name: X_test.astype(np.float32)})[0]
    print('shape ={0} and predicted probabilities={1}'.format(X_test.shape, pred_onx))
except (RuntimeError, InvalidArgument) as e:
    print("ERROR with Shape={0} - {1}".format(X_test.shape, e))

In [ ]:
preds = []
for i in pred_onx:
    preds.append(i[1])

print(preds)

## Predict Handler
For handling incoming json data

In [ ]:
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize  
import onnxruntime as rt
import warnings
warnings.filterwarnings('ignore')

def get_live_df(jsonfile):
    with open(jsonfile) as data_file:    
        data = json.load(data_file)  
    live_df = pd.io.json.json_normalize(data)
    live_df.rename(columns = {'body.xdmEntity._experienceplatform.identification.ecid' : 'ecid',
                          'body.xdmEntity.timestamp' : 'timestamp',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceAge' : 'age',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceBasicPrice' : 'pricequote',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceCarBrand' : 'carbrand',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceCarType' : 'cartype',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceCity' : 'city',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceCountry' : 'country',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceGender' : 'gender',
                          'body.xdmEntity._experienceplatform.carinsurance.insuranceLeasing' : 'leasing'}, inplace=True)
    return live_df

def age(x):
    if x < 20:
        return "u20"
    elif x > 19 and x < 29:
        return "20-28"
    elif x > 28 and x < 43:
        return "29-42"
    elif x > 42 and x < 55:
        return "43-54"
    elif x > 54 and x < 65:
        return "55-64"
    elif x >= 65: 
        return "65+"
    else: 
        return "Others"

def get_score_df(live_df):
    dfs = live_df[['ecid','timestamp', 'age', 'pricequote', 'carbrand', 'cartype', 'city', 'country', 'gender', 'leasing']]   
    # get values from loopkup table for matching mcids
    dfs['timestamp'] = pd.to_datetime(dfs['timestamp'])
    dfs['hour'] = dfs['timestamp'].dt.hour
    dfs['dayofweek'] = dfs['timestamp'].dt.dayofweek
    dfs['hour'] = dfs['hour'].astype(int)
    dfs['dayofweek'] = dfs['dayofweek'].astype(int)
    dfs.drop('timestamp', axis=1, inplace=True)

    dfs['age'] = dfs['age'].astype(int)
    dfs['age_bucket'] = dfs['age'].apply(lambda x: age(x))   
    dfs.drop('age', axis=1, inplace=True)
    
    cats = ['age_bucket', 'gender', 'city', 'country', 'carbrand', 'cartype', 'leasing']
    
    dfs = pd.get_dummies(dfs, columns = cats)
    
    X_score = dfs.drop('ecid', axis=1) 
    train_feats = ['hour', 'pricequote', 'age_bucket_29-42', 'age_bucket_43-54', 'age_bucket_55-64', 'gender_female', 'gender_male', 'city_Brussels', 'city_Zurich', 'dayofweek_0', 'dayofweek_1', 'dayofweek_2',
 'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6', 'country_australia', 'country_belgium', 'country_india', 'country_switzerland', 'carbrand_alfaromeo', 'carbrand_audi', 'carbrand_bmw',
 'carbrand_dodge', 'carbrand_saab', 'carbrand_volvo', 'cartype_convertible', 'cartype_hatchback', 'cartype_minivan', 'cartype_suv', 'leasing_no', 'leasing_yes']
    
    score_feats = X_score.columns.values.tolist()
    missing_feats = list(set(train_feats) - set(score_feats))
    extra_feats = list(set(score_feats) - set(train_feats))
    for c in extra_feats:
        X_score.drop(c, axis=1, inplace=True)
    for c in missing_feats:
        X_score[c] = 0
    X_score = X_score[train_feats]
    ecids = dfs['ecid'].tolist()
    
    return ecids, X_score

def predict(jsonfile):
    
    json_df = get_live_df(jsonfile)
    ecids, score_df = get_score_df(json_df)
    payload = np.array(score_df.values.tolist())
    
    sess = rt.InferenceSession("model_new.onnx")
    input_name = sess.get_inputs()[0].name
    prob_name = sess.get_outputs()[1].name
    pred_onx = sess.run([prob_name],{input_name: payload.astype(np.float32)})[0]
    
    preds = [i[1] for i in pred_onx]
    result = {}
    for i in range(len(ecids)):
        result[ecids[i]] = preds[i]
        
    return result

### predict Output of sample json feed

In [ ]:
output = predict('sample_demo.json')
output